# Task 2: Bayesian Change Point Analysis of Brent Oil Prices

This notebook implements a Bayesian change point detection model by importing the core logic from a separate Python script. This modular approach allows us to focus on data loading, model execution, and result interpretation in the notebook.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import arviz as az
import sys

# Define the absolute path to your project's root directory.
# ACTION: Replace this path with the output you got from the terminal.
project_root = r'D:\Project\BirhanEnergies_OilPriceAnalysis'

# Add the project root to the system path to allow imports from 'src'
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added '{project_root}' to system path.")

# Now, import the function. It should work correctly.
from src.models.bayesian_changepoint import run_changepoint_model

# Set plotting style
sns.set_style('whitegrid')

# Define the path to the processed data
processed_data_path = os.path.join(project_root, "data", "processed", "brent_oil_clean.csv")

try:
    df = pd.read_csv(processed_data_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    time_series_log_return = df['Log_Return'].dropna()
    data_to_model = time_series_log_return.values
    print("Data loaded successfully.")
    print("Log returns time series head:")
    print(time_series_log_return.head())
except FileNotFoundError:
    print("Error: Processed data not found. Please run the data cleaning script first.")
    time_series_log_return = None

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


Added 'D:\Project\BirhanEnergies_OilPriceAnalysis' to system path.
Data loaded successfully.
Log returns time series head:
Date
1987-05-21   -0.009709
1987-05-22    0.005405
1987-05-25    0.002692
1987-05-26    0.001612
1987-05-27   -0.001612
Name: Log_Return, dtype: float64


### 1. Bayesian Volatility Change Point Model Execution

We will now run the Bayesian model by calling the `run_changepoint_model` function from our script. This will perform the MCMC sampling and return the model's output.

In [ ]:
if time_series_log_return is not None:
    trace = run_changepoint_model(data_to_model)

    # Save the model output for future use (e.g., by the dashboard)
    model_output_path = os.path.join("..", "data", "processed", "changepoint_trace.nc")
    az.to_netcdf(trace, model_output_path)
    print(f"\nModel trace saved to {model_output_path}")
else:
    print("Skipping model creation as log return data is not available.")
    trace = None


Starting PyMC sampling for Bayesian Volatility Change Point Detection...


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [tau]
>NUTS: [sigma_1, sigma_2, mu_log_return]


d:\Project\BirhanEnergies_OilPriceAnalysis\venv\Lib\site-packages\rich\live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

d:\Project\BirhanEnergies_OilPriceAnalysis\venv\Lib\site-packages\pytensor\scalar\basic.py:2094: RuntimeWarning: 
divide by zero encountered in divide
  return x / y

d:\Project\BirhanEnergies_OilPriceAnalysis\venv\Lib\site-packages\pytensor\scalar\basic.py:2094: RuntimeWarning: 
overflow encountered in divide
  return x / y

d:\Project\BirhanEnergies_OilPriceAnalysis\venv\Lib\site-packages\pytensor\scalar\basic.py:3297: RuntimeWarning: 
overflow encountered in scalar multiply
  return x * x

### 2. Interpreting the Model Output

Now that the model has run, we can examine its posterior distributions to interpret the results. The summary table and trace plots help us check for model convergence and identify the most probable change point.

In [ ]:
if trace is not None:
    print("\nBayesian Volatility Model Summary:")
    display(az.summary(trace, var_names=["tau", "mu_log_return", "sigma_1", "sigma_2"]))

    tau_samples = trace.posterior["tau"].values.flatten()
    most_probable_tau_index = int(pd.Series(tau_samples).mode().iloc[0])
    most_probable_date = time_series_log_return.index[most_probable_tau_index].strftime('%Y-%m-%d')
    
    print(f"\nMost Probable Change Point Index: {most_probable_tau_index}")
    print(f"This corresponds to the date: {most_probable_date}")
else:
    print("No model trace to analyze.")

### 3. Visualizing the Change Point and Quantifying Impact

Finally, we'll visualize the detected change point and quantify the shift in volatility. We will use this information to associate the finding with a real-world event.

In [ ]:
if trace is not None and time_series_log_return is not None:
    plt.figure(figsize=(18, 8))
    plt.plot(time_series_log_return.index, time_series_log_return.values, label='Brent Oil Log Returns', color='blue', alpha=0.7)

    # Plot the most probable change point
    plt.axvline(x=time_series_log_return.index[most_probable_tau_index], color='green', linestyle='-', linewidth=2, label='Most Probable Volatility Change Point')
    
    plt.title('Bayesian Change Point Detection on Brent Oil Log Returns')
    plt.xlabel('Date')
    plt.ylabel('Log Return')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Quantify the impact
    posterior_sigma_1 = trace.posterior["sigma_1"].mean().item()
    posterior_sigma_2 = trace.posterior["sigma_2"].mean().item()
    percent_change_volatility = ((posterior_sigma_2 - posterior_sigma_1) / posterior_sigma_1) * 100 if posterior_sigma_1 != 0 else float('inf')

    print("\n--- Quantifying the Change Point Impact ---")
    print(f"Estimated Std Dev of Log Returns before Change (sigma_1): {posterior_sigma_1:.4f}")
    print(f"Estimated Std Dev of Log Returns after Change (sigma_2): {posterior_sigma_2:.4f}")
    print(f"Percentage change in volatility: {percent_change_volatility:.2f}%")
else:
    print("Skipping visualization and impact quantification.")